In [83]:
import torch
import torch.nn as nn
import torch.nn.functional as f


In [84]:
image_size = 64
latent_dim = 128
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [85]:
class generator(nn.Module):
    def __init__(self, latent_dim):
        super(generator, self).__init__()
        self.convtrans1 = nn.ConvTranspose2d(in_channels=latent_dim, out_channels=1024, kernel_size=4, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(1024)
        self.convtrans2 = nn.ConvTranspose2d(in_channels=1024 * 4 * 4, out_channels=512, kernel_size=4, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(512)
        self.covtrans3 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.convtrans4 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=4, stride=2, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.convtrans5 = nn.ConvTranspose2d(in_channels=128, out_channels=3, kernel_size=4, stride=2, padding=1)


    def forward(self, x):
        x = f.relu(self.bn1((self.convtrans1(x))))
        x = f.relu(self.bn2((self.convtrans2(x))))
        x = f.relu(self.bn3((self.convtrans3(x))))
        x = f.relu(self.bn4((self.convtrans4(x))))
        x = f.tanh(self.convtrans5(x))

        return x
        

In [86]:

generator = generator(latent_dim).to(device)
print(generator)

generator(
  (convtrans1): ConvTranspose2d(128, 1024, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (convtrans2): ConvTranspose2d(16384, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (covtrans3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (convtrans4): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (convtrans5): ConvTranspose2d(128, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
)


In [87]:
class discriminator(nn.Module):
    def __init__(self):
        super(discriminator, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=128, kernel_size=4, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(128)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=256, kernel_size=4, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(256)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(512)
        self.conv4 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=4, stride=2, padding=1)
        self.bn4 = nn.BatchNorm2d(1024)
        self.fc = nn.Linear(1024 * 4 * 4, 1)

    def forward(self, x):
        x = f.leaky_relu(self.conv1(x), negative_slope=0.2)
        x = f.leaky_relu(self.conv2(x), negative_slope=0.2)
        x = f.leaky_relu(self.conv3(x), negative_slope=0.2)
        x = f.leaky_relu(self.conv4(x), negative_slope=0.2)

        x = x.view(x.size(0),-1)
        x = f.sigmoid(self.fc(x))
        return x

In [88]:
discriminator = discriminator().to(device)
print(discriminator)

discriminator(
  (conv1): Conv2d(3, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(512, 1024, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn4): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=16384, out_features=1, bias=True)
)
